# Things to consider for continue
1. simulation with rankig over time

In [52]:
import networkx as nx 
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt
from datetime import datetime,timedelta

In [98]:
# dt = datetime.strptime('2020-04-01', '%Y-%m-%d')
dt = datetime.today()#.date()
str(dt)

'2022-05-13 18:37:57.259754'

In [99]:
def read_fights():
    path = r'C:\Users\z187900\Dropbox\Data projects\ufcstat_scrapy\all_fights_new.csv'
    df = pd.read_csv(path, parse_dates=True)[['Date', 'fighter1', 'fighter2', 'fighter1_result', 'fighter2_result', 'BELT', 'BOUNES', 'Weight', 'TIME_FORMAT', 'Method', 'Method_detail', 'Judge1_score', 'Judge2_score', 'Judge3_score', 'ROUND', 'TIME']]
    df = df[df['Date']!='Date']
    df.BOUNES = (df.BOUNES.str.split('/')).str[-1].str.split('.png').str[0]
    df.BELT = (df.BELT.str.split('/')).str[-1].str.split('.png').str[0]
    df.Weight = df.Weight.str.strip('Bout')
    df.Date = pd.to_datetime(df.Date.astype(str), infer_datetime_format=True)
    df.TIME_FORMAT = df.TIME_FORMAT.str.split(' ').str[0]#.astype('int')
    df = df[df['Date'] < str(dt)]
    return df

df = read_fights()

C:\Users\z187900\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning: Columns (19,22,28,29,31,37,38,40,46,47,49,55,56) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [100]:
def weight(w):
    if "Women's" in w:
        if 'Strawweight' in w:        new = "Women's Strawweight"
        elif "Flyweight" in w:        new = "Women'sFlyweight"
        elif "antamweight" in w:      new = "Women's Bantamweight"
        elif "Featherweight" in w:    new = "Women's Featherweight"
    elif "Flyweight" in w:         new = 'Flyweight'
    elif "antamweight" in w:       new = 'Bantamweight'
    elif "Featherweight" in w:     new = 'Featherweight'
    elif "Lightweight" in w:       new = 'Lightweight'
    elif "Welterweight" in w:      new = 'Welterweight'
    elif "Middleweight" in w:      new = 'Middleweight'
    elif "Light Heavyweight" in w: new = 'Light Heavyweight'
    elif "Heavyweight" in w:       new = 'Heavyweight'
    else: new = 'other'

    return new
df['Weight'] = df.apply(lambda x: weight(x.Weight), axis=1)


In [101]:
def fighter_perfomance(n):
    """
    n: number of years for performance rate
    """
    df_1 = df[(df['Date'] >= dt-timedelta(weeks=52*n))]\
                .groupby(['fighter1', 'fighter1_result','Weight' ])['TIME_FORMAT']\
                .count()\
                .unstack(level=-2)\
                .fillna(0)
    df_1.index.set_names(['fighter', 'weight'], inplace=True)
    df_2 = df[(df['Date'] >= dt-timedelta(weeks=52*n))]\
            .groupby(['fighter2', 'fighter2_result','Weight'])['TIME_FORMAT']\
            .count()\
            .unstack(level=-2)\
            .fillna(0)
    df_2.index.set_names(['fighter', 'weight'], inplace=True)
    df_perf = df_1.add(df_2, fill_value=0)
    df_perf['n_fights'] = df_perf.sum(axis=1)
    df_perf['perf_rat'] = (df_perf['n_fights'] -df_perf['L'])/df_perf['n_fights']
    df_perf.columns = df_perf.columns.values
    df_perf = df_perf.sort_values(by=['perf_rat', 'n_fights'], ascending=False).reset_index(level=-1)#[df_perf.n_fights>2]
    return df_perf
df_perf = fighter_perfomance(3)

In [102]:
# list(df['Weight'].value_counts().index)

In [103]:
# adjusts the winner and losser position
def win_lose(fighter1_result, fighter1, fighter2):
    if fighter1_result == 'W':
        return fighter1, fighter2
    elif fighter1_result == 'L':
        return fighter2, fighter1
    else:
        return fighter2, fighter1
df['winner'] = df.apply(lambda x: win_lose(x.fighter1_result, x.fighter1, x.fighter2), axis=1).str[0]
df['losser'] = df.apply(lambda x: win_lose(x.fighter1_result, x.fighter1, x.fighter2), axis=1).str[1]
df = df[['Date', 'winner', 'losser','BELT', 'BOUNES', 'Weight', 'TIME_FORMAT','Method', 'Method_detail',
         'Judge1_score', 'Judge2_score', 'Judge3_score', 'ROUND', 'TIME', 'fighter1', 'fighter2', 'fighter1_result', 'fighter2_result' ]]

In [104]:
# adjust Draws
def draw_adj(df):
    df_d = df[df['fighter1_result']=='D']

    df_d['w_trans'] = df_d['winner']
    df_d['winner'] = df_d['losser']
    df_d['losser'] = df_d['w_trans']
    df_d.drop('w_trans', axis=1, inplace=True)
    df = pd.concat([df, df_d], axis=0)
    return df
df = draw_adj(df)

C:\Users\z187900\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\z187900\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\z187900\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

In [105]:
# clean method
def method(m):
    if 'Decision' in m:
        new = 'Decision'
    elif 'KO' in m:
        new = 'KO'
    elif 'Submission' in m:
        new = 'sub'
    else:
        new = 'other'
    return new
df['method'] = df.apply(lambda x: method(x.Method), axis=1)

In [106]:
# assign values to method
def val(row):
    if row.fighter1_result =='D':
        new = 0
    elif row.fighter1_result =='NC':
        new = 0
    elif row.method == 'Decision':
        new = 5
    else:
        new = 7
    
    if row.BELT == 'belt':
        new = new*2
    return new
df['value'] = df.apply(lambda x: val(x), axis=1)

In [107]:
# recencey ration 
def recency(Date, dt):
    import math
    # ((dt - df['Date']).dt.days/365).apply(math.floor)
    date = math.floor((dt - Date).days/365)
    if date<3:
        val = 1
    elif (date>3 & date<10):
        val = 1 - ((date-2)/10)
    else:
        val = 0.1
    return val
df['adj'] = df.apply(lambda x: recency(x.Date, dt), axis=1)
df['adj_val'] = df['adj'] * df['value']

# Network analysis


In [108]:
df[df.duplicated(subset=['losser', 'winner'])]

,Date,winner,losser,BELT,BOUNES,Weight,TIME_FORMAT,Method,Method_detail,Judge1_score,...,ROUND,TIME,fighter1,fighter2,fighter1_result,fighter2_result,method,value,adj,adj_val
107,2020-02-29,Deiveson Figueiredo,Joseph Benavidez,NaN,NaN,Flyweight,5,KO/TKO,Punch to Head At Distance Non-title fight for ...,NaN,...,2,1:54,Joseph Benavidez,Deiveson Figueiredo,L,W,KO,7,1.0,7.0
281,1996-02-16,Don Frye,Gary Goodridge,belt,belt,other,1,KO/TKO,Punches to Head From Half Guard Submission to ...,NaN,...,1,2:14,Don Frye,Gary Goodridge,W,L,KO,14,-1.4,-19.6
287,1996-07-12,Don Frye,Mark Hall,NaN,NaN,other,1,KO/TKO,Headbutt to Head From Guard,NaN,...,1,10:21,Don Frye,Mark Hall,W,L,KO,7,-1.3,-9.1
299,1995-04-07,Dan Severn,Oleg Taktarov,NaN,NaN,other,1,TKO - Doctor's Stoppage,Facial Cut,NaN,...,1,4:21,Dan Severn,Oleg Taktarov,W,L,KO,7,-1.5,-10.5
332,1995-07-14,Oleg Taktarov,Dave Beneteau,NaN,NaN,other,1,Submission,Guillotine Choke After Drop to Guard,NaN,...,1,0:57,Oleg Taktarov,Dave Beneteau,W,L,sub,7,-1.4,-9.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12040,2020-10-24,Da-Un Jung,Sam Alvey,NaN,NaN,Light Heavyweight,3,Decision - Split,NaN,28 - 29.,...,3,5:00,Da-Un Jung,Sam Alvey,D,D,Decision,0,1.0,0.0
12091,2020-12-12,Deiveson Figueiredo,Brandon Moreno,belt,fight,Flyweight,5,Decision - Majority,NaN,46 - 48.,...,5,5:00,Deiveson Figueiredo,Brandon Moreno,D,D,Decision,0,1.0,0.0
12133,2020-10-03,Charles Jourdain,Josh Culibao,NaN,NaN,Featherweight,3,Decision - Split,NaN,27 - 30.,...,3,5:00,Charles Jourdain,Josh Culibao,D,D,Decision,0,1.0,0.0
12258,2021-05-01,Ion Cutelaba,Dustin Jacoby,NaN,NaN,Light Heavyweight,3,Decision - Split,NaN,28 - 29.,...,3,5:00,Ion Cutelaba,Dustin Jacoby,D,D,Decision,0,1.0,0.0


In [109]:
G_weighted = nx.from_pandas_edgelist(df, 'losser', 'winner', create_using=nx.DiGraph, edge_attr=['adj_val', 'Date'])

weighted_pagerank = nx.pagerank(G_weighted, alpha=0.95)

df_ranked = pd.DataFrame.from_dict(weighted_pagerank, orient='index')
df_ranked.columns = ['page_rank']
df_ranked.sort_values(by=['page_rank'], ascending=False, inplace=True)



In [116]:
weight_list = df['Weight'].unique()
weight_list

array(['Welterweight', 'Middleweight', 'Featherweight', 'Bantamweight',
       "Women's Bantamweight", "Women'sFlyweight", 'Flyweight',
       'Heavyweight', 'Lightweight', 'other', "Women's Strawweight",
       'Light Heavyweight', "Women's Featherweight"], dtype=object)

In [111]:
df_w_rank_all = pd.DataFrame()
for weight_to_show in weight_list:
    cond = (df['Weight'] == weight_to_show) & (df['Date'] >= dt-timedelta(weeks=52*2))
    fighters_list = set(list(df[cond]['winner'])+list(df[cond]['losser']))
    df_w_rank = df_ranked[df_ranked.index.isin(fighters_list)].sort_values(by=['page_rank'], ascending=False)

    df_perf_select = df_perf[(df_perf.index.isin(fighters_list)) & (df_perf['weight']==weight_to_show)].sort_values('perf_rat', ascending=False)[['perf_rat', 'n_fights']].reset_index()
    df_w_rank_select = df_w_rank.reset_index().rename({'index':'fighter'}, axis=1)

    df_w_rank_adj = pd.merge(df_w_rank_select, df_perf_select, on='fighter', how='inner')#.rename({'key_0':'fighter'}, axis=1)
    df_w_rank_adj['adj_p_rank'] = df_w_rank_adj['page_rank']*df_w_rank_adj['perf_rat']
    df_w_rank_adj.sort_values(by='adj_p_rank', ascending=False, inplace=True,ignore_index=True)
    df_w_rank_adj['weight'] = weight_to_show
    df_w_rank_all = pd.concat([df_w_rank_all,df_w_rank_adj])

In [112]:
# df_w_rank_all[(df_w_rank_all['n_fights']>1)].to_csv('w_ranking.csv')
df_w_rank_all.to_csv('w_ranking.csv')

In [117]:
df_w_rank_all[(df_w_rank_all['n_fights']>1) & (df_w_rank_all['weight'] == 'Lightweight')].head(25)

,fighter,page_rank,perf_rat,n_fights,adj_p_rank,weight
0,Charles Oliveira,0.004040,1.000000,11.0,0.004040,Lightweight
1,Rafael Dos Anjos,0.003855,1.000000,2.0,0.003855,Lightweight
2,Khabib Nurmagomedov,0.003664,1.000000,4.0,0.003664,Lightweight
3,Beneil Dariush,0.003152,1.000000,9.0,0.003152,Lightweight
4,Dustin Poirier,0.004792,0.625000,8.0,0.002995,Lightweight
5,Islam Makhachev,0.001756,1.000000,6.0,0.001756,Lightweight
6,Ricky Glenn,0.001711,1.000000,2.0,0.001711,Lightweight
7,Rafael Fiziev,0.001466,1.000000,8.0,0.001466,Lightweight
8,Grant Dawson,0.001369,1.000000,4.0,0.001369,Lightweight
9,Francisco Trinaldo,0.001555,0.750000,8.0,0.001166,Lightweight


In [114]:
def pfp_performance(m):
    df_1 = df[(df['Date'] >= dt-timedelta(weeks=52*m))].groupby(['fighter1', 'fighter1_result'])['Weight'].count().unstack(level=-1).fillna(0)
    df_1.index.set_names(['fighter'], inplace=True)
    df_2 = df[(df['Date'] >= dt-timedelta(weeks=52*m))].groupby(['fighter2', 'fighter2_result'])['Weight'].count().unstack(level=-1).fillna(0)
    df_2.index.set_names(['fighter'], inplace=True)
    df_perf = df_1.add(df_2, fill_value=0)
    df_perf['n_fights'] = df_perf.sum(axis=1)
    df_perf['perf_rat'] = (df_perf['n_fights'] -df_perf['L'])/df_perf['n_fights']
    df_perf.columns = df_perf.columns.values
    df_perf = df_perf.sort_values(by=['perf_rat', 'n_fights'], ascending=False).reset_index(level=-1)#[df_perf.n_fights>2]
    return df_perf



In [115]:
df_perf_pfp = pfp_performance(3)
df_pfp = pd.merge(df_ranked.reset_index().rename({'index': 'fighter'}, axis=1), df_perf_pfp, on='fighter', how='inner')
df_pfp['adj_p_rank'] = df_pfp['page_rank'] * df_pfp['perf_rat']
df_pfp.sort_values(by='adj_p_rank', ascending=False, inplace=True,ignore_index=True)
# df_pfp[(df_pfp['n_fights']>1)].to_csv('pfp_ranking.csv')
df_pfp.to_csv('pfp_ranking.csv')
df_pfp[(df_pfp['n_fights']>1)].head(25)


,fighter,page_rank,D,L,NC,W,n_fights,perf_rat,adj_p_rank
0,Amanda Nunes,0.016238,0.0,1.0,0.0,8.0,9.0,0.888889,0.014434
1,Francis Ngannou,0.010725,0.0,0.0,0.0,7.0,7.0,1.000000,0.010725
2,Jon Jones,0.009362,0.0,0.0,0.0,4.0,4.0,1.000000,0.009362
3,Brandon Moreno,0.007937,8.0,1.0,0.0,7.0,16.0,0.937500,0.007441
4,Derrick Lewis,0.009092,0.0,2.0,0.0,9.0,11.0,0.818182,0.007439
5,Stipe Miocic,0.011097,0.0,2.0,0.0,4.0,6.0,0.666667,0.007398
6,Julianna Pena,0.009810,0.0,2.0,0.0,5.0,7.0,0.714286,0.007007
7,Henry Cejudo,0.006662,0.0,0.0,0.0,4.0,4.0,1.000000,0.006662
8,Valentina Shevchenko,0.006565,0.0,0.0,0.0,11.0,11.0,1.000000,0.006565
9,Jan Blachowicz,0.007023,0.0,1.0,0.0,10.0,11.0,0.909091,0.006384
